In [3]:
import galois
from collections import Counter

# Define GF(2)
GF2 = galois.GF(2)

def F_0(x0, x1, x2):
    return GF2(0)

def F_1(x0, x1, x2):
    return GF2(0)

def F_2(x0, x1, x2):
    return x0 * x1

def product(u, v):
    u0, u1, u2 = u
    v0, v1, v2 = v
    return (
        u0 * v0 + u2 * v1 + u1 * v2 + u2 * v2,
        u0 * v1 + u1 * v0 + u2 * v2,
        u1 * v1 + u2 * v0 + u0 * v2 + u2 * v1 + u1 * v2 + u2 * v2
    )

def compute_c_derivative(xa, x, c0, c1, c2):
    x_a0, x_a1, x_a2 = GF2(xa[0]), GF2(xa[1]), GF2(xa[2])
    x0, x1, x2 = GF2(x[0]), GF2(x[1]), GF2(x[2])
    c0, c1, c2 = GF2(c0), GF2(c1), GF2(c2)

    Fx_a0 = F_0(x_a0, x_a1, x_a2)
    Fx_a1 = F_1(x_a0, x_a1, x_a2)
    Fx_a2 = F_2(x_a0, x_a1, x_a2)

    c_Fx0, c_Fx1, c_Fx2 = product((c0, c1, c2), (F_0(x0, x1, x2), F_1(x0, x1, x2), F_2(x0, x1, x2)))

    D_c_F0 = Fx_a0 + c_Fx0
    D_c_F1 = Fx_a1 + c_Fx1
    D_c_F2 = Fx_a2 + c_Fx2

    return (int(D_c_F0), int(D_c_F1), int(D_c_F2))

# Generate all possible values
a_values = [(GF2(a0), GF2(a1), GF2(a2)) for a0 in range(2) for a1 in range(2) for a2 in range(2)]
x_values = [(GF2(x0), GF2(x1), GF2(x2)) for x0 in range(2) for x1 in range(2) for x2 in range(2)]
c_values = [(GF2(c0), GF2(c1), GF2(c2)) for c0 in range(2) for c1 in range(2) for c2 in range(2)]

# Counters
results = []
total_counter = Counter()
a_counters = {tuple(map(int, a)): Counter() for a in a_values}
c_counters = {tuple(map(int, c)): Counter() for c in c_values}
a_c_counters = {tuple(map(int, a)): {tuple(map(int, c)): Counter() for c in c_values} for a in a_values}

# Main computation
for a in a_values:
    a_tuple = tuple(map(int, a))
    for x in x_values:
        x_tuple = tuple(map(int, x))
        x_a = (int(x[0] + a[0]), int(x[1] + a[1]), int(x[2] + a[2]))
        for c in c_values:
            c_tuple = tuple(map(int, c))
            D_c_F = compute_c_derivative(x_a, x, *c)
            results.append((c_tuple, a_tuple, x_tuple, D_c_F))
            a_counters[a_tuple][D_c_F] += 1
            c_counters[c_tuple][D_c_F] += 1
            a_c_counters[a_tuple][c_tuple][D_c_F] += 1
            total_counter[D_c_F] += 1

# Sort results by c -> a -> x
results.sort()

# Display results
print("\n## Final Results (binary):")
previous_a = None
for c_tuple, a_tuple, x_tuple, (expr1, expr2, expr3) in results:
    if previous_a is not None and previous_a != a_tuple:
        print()
    print(f"c = {c_tuple}, a = {a_tuple}, x = {x_tuple}: (D_c_F0 = {expr1}, D_c_F1 = {expr2}, D_c_F2 = {expr3})")
    previous_a = a_tuple

# Display count of different results per a
print("\n## Result Counts Per a:")
for a, counter in sorted(a_counters.items()):
    print(f"For a = {a}:")
    for key, count in sorted(counter.items()):
        print(f"  Result {key}: {count} occurrences")
    print()

# Display count of different results per c
print("\n## Result Counts Per c:")
for c, counter in sorted(c_counters.items()):
    print(f"For c = {c}:")
    for key, count in sorted(counter.items()):
        print(f"  Result {key}: {count} occurrences")
    print()

# Display count per (a, c)
print("\n## Result Counts Per (a, c):")
for a, c_dict in sorted(a_c_counters.items()):
    print(f"For a = {a}:")
    for c, counter in sorted(c_dict.items()):
        print(f"  For c = {c}:")
        for key, count in sorted(counter.items()):
            print(f"    Result {key}: {count} occurrences")
    print()

# Display total count
print("\n## Total Result Counts:")
for key, count in sorted(total_counter.items()):
    print(f"Result {key}: {count} occurrences")



## Result Counts Per a:
For a = (0, 0, 0):
  Result (0, 0, 0): 50 occurrences
  Result (0, 0, 1): 2 occurrences
  Result (0, 1, 0): 2 occurrences
  Result (0, 1, 1): 2 occurrences
  Result (1, 0, 0): 2 occurrences
  Result (1, 0, 1): 2 occurrences
  Result (1, 1, 0): 2 occurrences
  Result (1, 1, 1): 2 occurrences

For a = (0, 0, 1):
  Result (0, 0, 0): 50 occurrences
  Result (0, 0, 1): 2 occurrences
  Result (0, 1, 0): 2 occurrences
  Result (0, 1, 1): 2 occurrences
  Result (1, 0, 0): 2 occurrences
  Result (1, 0, 1): 2 occurrences
  Result (1, 1, 0): 2 occurrences
  Result (1, 1, 1): 2 occurrences

For a = (0, 1, 0):
  Result (0, 0, 0): 34 occurrences
  Result (0, 0, 1): 18 occurrences
  Result (0, 1, 0): 2 occurrences
  Result (0, 1, 1): 2 occurrences
  Result (1, 0, 0): 2 occurrences
  Result (1, 0, 1): 2 occurrences
  Result (1, 1, 0): 2 occurrences
  Result (1, 1, 1): 2 occurrences

For a = (0, 1, 1):
  Result (0, 0, 0): 34 occurrences
  Result (0, 0, 1): 18 occurrences
  Resu